## Prediction
If the average lifetime of a customer is 90 days since their first deposit :  
build a model to predict the Lifetime Value of a customer given his first 15 days of activity. What other data could be pertinent to answer this question? Given the data provided is the assumption of 90 days valid? 

In [115]:
import pandas as pd
import pandas_profiling
import matplotlib
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans

pd.options.display.max_columns = None

matplotlib.style.use('ggplot')
%matplotlib inline

In [116]:
# get dataset
df_bc = pd.read_csv('data/dataset_betclic.csv')

# convert string date to date
df_bc['BirthDate'] = pd.to_datetime(df_bc['BirthDate'], format='%Y-%m-%dT%H:%M:%S.%fZ')
df_bc['FirstDepositDate'] = pd.to_datetime(df_bc['FirstDepositDate'], format='%Y-%m-%dT%H:%M:%S.%fZ')
df_bc['BetDate'] = pd.to_datetime(df_bc['BetDate'], format='%Y%m%d')

## Variables

### Lifetime

The lifetime of a customer is the difference between the date of the last bet and the first deposit date  
$max(BetDate) - min(FirstDepositDate)$

In [117]:
df_max_BetDate = df_bc.groupby(['UserId'])['BetDate'].max().reset_index()
df_max_BetDate.columns = ['UserId', 'max_BetDate']

df_min_FirstDepositDate = df_bc.groupby(['UserId'])['FirstDepositDate'].min().reset_index()
df_min_FirstDepositDate.columns = ['UserId', 'min_FirstDepositDate']

df_ltv = pd.merge(df_bc, 
                  df_max_BetDate, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])

df_ltv = pd.merge(df_ltv, 
                  df_min_FirstDepositDate, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])

df_ltv['lifetime'] = df_ltv['max_BetDate'] - df_ltv['min_FirstDepositDate']

df_ltv.head()

,UserId,Country,Gender,BirthDate,PartnerType,FirstDepositDate,BetDate,BetId,Application,ProductName,IsLive,BetAmount,AmountWon,Odds,Cashout,max_BetDate,min_FirstDepositDate,lifetime
0,10783501,Portugal,M,1992-06-08,PPC,2017-01-03,2017-01-03,607023702,BETCLIC.PT,SportsBook,False,5.00,0.0,124.76,0.0,2017-09-10,2017-01-03,250 days
1,10782912,Ivory Coast,M,1996-03-18,Internal,2017-01-03,2017-01-03,607023878,BETCLIC.COM,SportsBook,False,5.00,23.0,4.60,0.0,2017-10-21,2017-01-03,291 days
2,10780813,France,F,1998-04-05,PPC,2017-01-01,2017-01-03,607024082,BETCLIC.FR,SportsBook,False,5.00,0.0,8.00,0.0,2017-09-04,2017-01-01,246 days
3,10783215,Portugal,M,1976-12-17,Internal,2017-01-03,2017-01-03,607024088,BETCLIC.PT,SportsBook,False,1.00,0.0,41.08,0.0,2017-11-19,2017-01-03,320 days
4,10783401,Poland,M,1995-04-16,Affiliation,2017-01-03,2017-01-03,607024113,EXPEKT.COM,SportsBook,False,5.67,0.0,3.75,0.0,2017-01-09,2017-01-03,6 days


We check if the average lifetime of a customer given by Betclick, 90 jours, is the same in our data :

In [118]:
df_ltv.drop_duplicates(subset='UserId')['lifetime'].mean()

Timedelta('98 days 01:44:26.035675')

We find an average at 98 days

### Value

I consider the Value of a customer the amount he lost during his active lifetime on Betclic  
$BetAmount - AmountWon$

In [119]:
df_bet_amount = df_bc.groupby(['UserId'])['BetAmount'].sum().reset_index()
df_bet_amount.columns = ['UserId', 'tot_bet_amount']

df_ltv = pd.merge(df_ltv, 
                  df_bet_amount, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])

df_amount_won = df_bc.groupby(['UserId'])['AmountWon'].sum().reset_index()
df_amount_won.columns = ['UserId', 'tot_amount_won']

df_ltv = pd.merge(df_ltv, 
                  df_amount_won, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])

df_ltv['value_brut'] = df_ltv['tot_bet_amount'] - df_ltv['tot_amount_won']

df_ltv.head()

,UserId,Country,Gender,BirthDate,PartnerType,FirstDepositDate,BetDate,BetId,Application,ProductName,IsLive,BetAmount,AmountWon,Odds,Cashout,max_BetDate,min_FirstDepositDate,lifetime,tot_bet_amount,tot_amount_won,value_brut
0,10783501,Portugal,M,1992-06-08,PPC,2017-01-03,2017-01-03,607023702,BETCLIC.PT,SportsBook,False,5.00,0.0,124.76,0.0,2017-09-10,2017-01-03,250 days,30.00,0.00,30.00
1,10782912,Ivory Coast,M,1996-03-18,Internal,2017-01-03,2017-01-03,607023878,BETCLIC.COM,SportsBook,False,5.00,23.0,4.60,0.0,2017-10-21,2017-01-03,291 days,1044.00,900.16,143.84
2,10780813,France,F,1998-04-05,PPC,2017-01-01,2017-01-03,607024082,BETCLIC.FR,SportsBook,False,5.00,0.0,8.00,0.0,2017-09-04,2017-01-01,246 days,966.71,481.26,485.45
3,10783215,Portugal,M,1976-12-17,Internal,2017-01-03,2017-01-03,607024088,BETCLIC.PT,SportsBook,False,1.00,0.0,41.08,0.0,2017-11-19,2017-01-03,320 days,503.30,320.30,183.00
4,10783401,Poland,M,1995-04-16,Affiliation,2017-01-03,2017-01-03,607024113,EXPEKT.COM,SportsBook,False,5.67,0.0,3.75,0.0,2017-01-09,2017-01-03,6 days,186.77,169.85,16.92


## Lifetime Value
I consider the lifetime value as :  
$\frac{Value}{Lifetime}$  
I think this value is more interesting than :  
$BetAmount - AmountWon$  
as it allows us to compare two customers independantly of the time spend on the website. 

In [120]:
# value/lifetime
df_ltv['lifetime_int'] = df_ltv['lifetime'].astype('<m8[D]')
# turn 1 the 0 lifetime
df_ltv['lifetime_int'] = np.where(df_ltv['lifetime_int'] == 0, 1, df_ltv['lifetime_int'])
df_ltv['lifetime_value'] = df_ltv['value_brut'] / df_ltv['lifetime_int']

In [121]:
df_ltv.head()

,UserId,Country,Gender,BirthDate,PartnerType,FirstDepositDate,BetDate,BetId,Application,ProductName,IsLive,BetAmount,AmountWon,Odds,Cashout,max_BetDate,min_FirstDepositDate,lifetime,tot_bet_amount,tot_amount_won,value_brut,lifetime_int,lifetime_value
0,10783501,Portugal,M,1992-06-08,PPC,2017-01-03,2017-01-03,607023702,BETCLIC.PT,SportsBook,False,5.00,0.0,124.76,0.0,2017-09-10,2017-01-03,250 days,30.00,0.00,30.00,250.0,0.120000
1,10782912,Ivory Coast,M,1996-03-18,Internal,2017-01-03,2017-01-03,607023878,BETCLIC.COM,SportsBook,False,5.00,23.0,4.60,0.0,2017-10-21,2017-01-03,291 days,1044.00,900.16,143.84,291.0,0.494296
2,10780813,France,F,1998-04-05,PPC,2017-01-01,2017-01-03,607024082,BETCLIC.FR,SportsBook,False,5.00,0.0,8.00,0.0,2017-09-04,2017-01-01,246 days,966.71,481.26,485.45,246.0,1.973374
3,10783215,Portugal,M,1976-12-17,Internal,2017-01-03,2017-01-03,607024088,BETCLIC.PT,SportsBook,False,1.00,0.0,41.08,0.0,2017-11-19,2017-01-03,320 days,503.30,320.30,183.00,320.0,0.571875
4,10783401,Poland,M,1995-04-16,Affiliation,2017-01-03,2017-01-03,607024113,EXPEKT.COM,SportsBook,False,5.67,0.0,3.75,0.0,2017-01-09,2017-01-03,6 days,186.77,169.85,16.92,6.0,2.820000


In [122]:
# age
now = pd.Timestamp(datetime.now())
df_ltv['age'] = (now - df_bc['BirthDate']).astype('<m8[Y]')

## 15 jours

In [123]:
df_ltv_15 = df_ltv[df_ltv['BetDate'] <= (df_ltv['FirstDepositDate'] + timedelta(days=15))]

### Building feature

#### Features on bets

In [124]:
mean_bet_by_user = df_ltv_15.groupby('UserId')['BetAmount'].mean().reset_index()
mean_bet_by_user.columns = ['UserId', 'mean_bet']

min_bet_by_user = df_ltv_15.groupby('UserId')['BetAmount'].min().reset_index()
min_bet_by_user.columns = ['UserId', 'min_bet']

max_bet_by_user = df_ltv_15.groupby('UserId')['BetAmount'].max().reset_index()
max_bet_by_user.columns = ['UserId', 'max_bet']

median_bet_by_user = df_ltv_15.groupby('UserId')['BetAmount'].median().reset_index()
median_bet_by_user.columns = ['UserId', 'median_bet']

sum_bet_by_user = df_ltv_15.groupby('UserId')['BetAmount'].sum().reset_index()
sum_bet_by_user.columns = ['UserId', 'sum_bet']

count_bet_by_user = df_ltv_15.groupby('UserId')['BetId'].count().reset_index()
count_bet_by_user.columns = ['UserId', 'count_bet']


#### Feature on bet won

In [125]:
mean_bet_won_by_user = df_ltv_15.groupby('UserId')['AmountWon'].mean().reset_index()
mean_bet_won_by_user.columns = ['UserId', 'mean_bet_won']

min_bet_won_by_user = df_ltv_15.groupby('UserId')['AmountWon'].min().reset_index()
min_bet_won_by_user.columns = ['UserId', 'min_bet_won']

max_bet_won_by_user = df_ltv_15.groupby('UserId')['AmountWon'].max().reset_index()
max_bet_won_by_user.columns = ['UserId', 'max_bet_won']

median_bet_won_by_user = df_ltv_15.groupby('UserId')['AmountWon'].median().reset_index()
median_bet_won_by_user.columns = ['UserId', 'median_bet_won']

sum_bet_won_by_user = df_ltv_15.groupby('UserId')['AmountWon'].sum().reset_index()
sum_bet_won_by_user.columns = ['UserId', 'sum_bet_won']

count_bet_won_by_user = df_ltv_15[df_ltv_15['AmountWon']>0].groupby('UserId')['BetId'].count().reset_index()
count_bet_won_by_user.columns = ['UserId', 'count_bet_won']

#### Other feature

In [126]:
mean_ods_by_user = df_ltv_15.groupby('UserId')['Odds'].mean()

In [127]:
islive_by_user = df_ltv_15.groupby('UserId')['IsLive'].max()

#### Categorical features

In [128]:
application_used = pd.get_dummies(df_ltv_15[['UserId', 'Application']]).groupby('UserId').sum()

partnertype_used = pd.get_dummies(df_ltv_15[['UserId', 'PartnerType']]).groupby('UserId').sum()

gender_used = pd.get_dummies(df_ltv_15[['UserId', 'Gender']]).groupby('UserId').sum()
gender_used = gender_used / gender_used
gender_used = gender_used.fillna(0)

Country_used = pd.get_dummies(df_ltv_15[['UserId', 'Country']]).groupby('UserId').sum()
Country_used = Country_used / Country_used
Country_used = Country_used.fillna(0)

### Merging all features

In [129]:
df_res = pd.merge(mean_bet_by_user, 
                  min_bet_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  max_bet_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  median_bet_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  sum_bet_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  count_bet_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  mean_bet_won_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  min_bet_won_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  max_bet_won_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  median_bet_won_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  sum_bet_won_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  count_bet_won_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  application_used, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  mean_ods_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  islive_by_user, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  partnertype_used, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  gender_used, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])
df_res = pd.merge(df_res, 
                  Country_used, 
                  how='left', 
                  left_on=['UserId'], 
                  right_on=['UserId'])

#### Adding interesting feature

In [130]:
df_res['ratio_win'] = df_res['count_bet_won'] / df_res['count_bet']

### Final cleaning

In [131]:
df_res = df_res.fillna(0)

In [132]:
ltv_age = df_ltv[['UserId', 'age']].drop_duplicates()

In [133]:
df_final = pd.merge(df_res, 
                    ltv_age, 
                    how='left', 
                    left_on=['UserId'], 
                    right_on=['UserId'])

In [134]:
ltv_to_evaluation = df_ltv[['UserId', 'lifetime_value']].drop_duplicates()

In [135]:
df_final = pd.merge(df_res, 
                    ltv_to_evaluation, 
                    how='left', 
                    left_on=['UserId'], 
                    right_on=['UserId'])

#### Saving the dataset

In [136]:
df_final.to_csv('15_bc.csv', index=None, sep=';')

# Clustering
N'etant pas satisfait de l'approche du probleme en regression (code dans le notebook regression.ipynb), je propose une autre facon de voir les choses.  
En clusterisant les comportements, nous obtenons une vision des differents profils des clients et ainsi observer les differences de lifetime value de chaque groupe.  
Le choix de 5 classes est arbitraire (ouvert à discussion et à une analyse plus poussée).

## Kmeans #1

In [137]:
# compute kmeans
RANDOM_STATE = 50
NB_CLUSTERS = 5
X = df_final[list(set(df_final).difference({'UserId'}))]
kmeans = KMeans(n_clusters=NB_CLUSTERS,
                random_state=RANDOM_STATE, n_jobs=8)
kmeans.fit(X)
X['kmeans_cluster'] = kmeans.labels_

In [138]:
from matplotlib import colors as matplot_colors
import matplotlib.pyplot as plt

def background_gradient(row):
    cmap = 'YlGnBu'
    m = row.min()
    M = row.max()
    low = 0
    high = 0.2
    rng = M - m
    norm = matplot_colors.Normalize(m - (rng * low), M + (rng * high))
    normed = norm(list(row))
    c = [matplot_colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]


X.groupby('kmeans_cluster').mean().T.style.apply(background_gradient, axis=1)

kmeans_cluster,0,1,2,3,4
min_bet,20.9906,44.85,172.86,14.2054,17.2431
median_bet,26.2083,1118,898.082,129.817,68.6359
Country_France,0.525399,0,0.666667,0.521739,0.555046
Country_Netherlands,0.00178389,0,0,0,0.00458716
PartnerType_Ad Networks,0.0155878,0,0,0,0
PartnerType_Sponsoring,0.0133367,0,0,0,0
Application_BETCLIC.PT,6.49291,0,0,13.9565,17.4472
min_bet_won,10.9776,0,0,0,11.578
Country_Hungary,0.00552158,0,0,0,0.00688073
sum_bet_won,96.3705,178384,46857.9,12045.7,2421.86


In [139]:
X.groupby('kmeans_cluster').count()

,min_bet,median_bet,Country_France,Country_Netherlands,PartnerType_Ad Networks,PartnerType_Sponsoring,Application_BETCLIC.PT,min_bet_won,Country_Hungary,sum_bet_won,Gender_F,mean_bet,PartnerType_Expekt,Country_United Kingdom,Odds,Country_Portugal,PartnerType_Referral,lifetime_value,PartnerType_betclic,Country_Norway,mean_bet_won,count_bet,PartnerType_Emailing,Country_Italy,ratio_win,Country_Ivory Coast,PartnerType_Affiliation,Application_BETCLIC.COM,Application_BETCLIC.IT,max_bet_won,max_bet,Application_BETCLIC.FR,sum_bet,Country_Finland,Country_Sweden,PartnerType_Offline,PartnerType_SEO,Country_Germany,Country_Poland,IsLive,PartnerType_Mobile,PartnerType_PPC,Gender_M,count_bet_won,Country_Switzerland,PartnerType_Partnerships,PartnerType_Internal,Application_EXPEKT.COM,median_bet_won
kmeans_cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544,23544
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
3,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
4,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436,436


### Conclusion Kmeans #1
Les clusters sont beaucoup trop desequilibrés, les valeurs 'financieres' comportent des cas extremes qui polluent le bon placement des centroides.

## Kmeans #2 : Robust scaler

In [146]:
X = df_final[list(set(df_final).difference({'UserId'}))]

transformer = RobustScaler().fit(X)
df_robust = pd.DataFrame(transformer.transform(X))
df_robust.columns = X.columns
df_robust#['mean_bet'].hist()

# compute kmeans
RANDOM_STATE = 50
NB_CLUSTERS = 5
X = df_robust[list(set(df_robust).difference({'UserId'}))]
kmeans = KMeans(n_clusters=NB_CLUSTERS,
                random_state=RANDOM_STATE, n_jobs=8)
kmeans.fit(X)
X['kmeans_cluster'] = kmeans.labels_
X.groupby('kmeans_cluster').mean().T.style.apply(background_gradient, axis=1)

kmeans_cluster,0,1,2,3,4
min_bet,0.801304,7.32414,4.4175,-0.05431,2.36657
median_bet,0.484519,4.81751,2.6296,-0.12915,1.19718
Country_France,-0.479141,-0.28169,-0.323034,-0.511111,-1
Country_Netherlands,0.00185697,0,0.00140449,0,0
PartnerType_Ad Networks,0.015849,0,0,0,0
PartnerType_Sponsoring,0.0135602,0,0,0,0
Application_BETCLIC.PT,6.92274,0.0140845,0.00561798,9,0
min_bet_won,1.18783,765.946,248.857,0,4429.5
Country_Hungary,0.00574365,0,0,0,0
sum_bet_won,1.23799,12.0724,4.01914,-0.00827777,37.9947


In [149]:
X.groupby('kmeans_cluster').count()

,min_bet,median_bet,Country_France,Country_Netherlands,PartnerType_Ad Networks,PartnerType_Sponsoring,Application_BETCLIC.PT,min_bet_won,Country_Hungary,sum_bet_won,Application_EXPEKT.COM,Gender_F,PartnerType_Expekt,Odds,Country_Portugal,PartnerType_Referral,lifetime_value,PartnerType_betclic,Country_Norway,mean_bet_won,count_bet,PartnerType_Emailing,Country_Italy,ratio_win,Country_Ivory Coast,PartnerType_Affiliation,median_bet_won,Application_BETCLIC.COM,Application_BETCLIC.IT,max_bet_won,max_bet,Application_BETCLIC.FR,sum_bet,Country_Finland,Country_Sweden,PartnerType_Offline,PartnerType_SEO,Country_Germany,Country_Poland,IsLive,PartnerType_Mobile,PartnerType_PPC,Gender_M,count_bet_won,Country_Switzerland,PartnerType_Partnerships,PartnerType_Internal,mean_bet,Country_United Kingdom
kmeans_cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156,23156
1,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71
2,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712,712
3,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


### Conlusion Kmeans #2
Amelioration du kmeans #1 mais toujours aussi desequilibré, essayons avec un scaler plus classique.

## Kmeans #3

In [142]:
X = df_final[list(set(df_final).difference({'UserId'}))]

transformer = MinMaxScaler().fit(X)
df_robust = pd.DataFrame(transformer.transform(X))
df_robust.columns = X.columns
df_robust#['mean_bet'].hist()

# compute kmeans
RANDOM_STATE = 50
NB_CLUSTERS = 5
X = df_robust[list(set(df_robust).difference({'UserId'}))]
kmeans = KMeans(n_clusters=NB_CLUSTERS,
                random_state=RANDOM_STATE, n_jobs=8)
kmeans.fit(X)
df_final['kmeans_cluster'] = kmeans.labels_
df_final.groupby('kmeans_cluster').mean().T.style.apply(background_gradient, axis=1)

/home/tom/source/smart_click/betclic/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


kmeans_cluster,0,1,2,3,4
UserId,1.06828e+07,1.07012e+07,1.07255e+07,1.0637e+07,1.07549e+07
mean_bet,47.9892,12.3854,26.7269,16.6599,42.6891
min_bet,39.1547,5.23521,20.646,4.98034,34.5676
max_bet,64.4959,31.2158,40.4294,59.5305,60.2931
median_bet,46.0023,10.6933,25.1792,12.9131,40.697
sum_bet,111.263,264.265,167.534,355.074,154.506
count_bet,6.62808,37.3089,13.2866,32.2382,14.7797
mean_bet_won,32.8917,8.09162,20.5327,10.6705,30.794
min_bet_won,22.7138,0.63661,10.6021,0.440544,20.6537
max_bet_won,57.3083,46.0442,46.7482,78.3702,62.097


In [152]:
df_final.groupby('kmeans_cluster').count()['UserId']

kmeans_cluster
0    6945
1    5386
2    2917
3    5441
4    3341
Name: UserId, dtype: int64

### Conclusion kmeans #3
Les clusters sont bien equilibrés et il est possible de raconter une histoire sur le cluster ayant une plus forte lifetime value:  
Les clients generant un $\text{lifetime value}$ haut parient des montants elevés en moyenne, jouent assez peu souvant, sont des femmes, francaise/italienne, sur des Odds assez bas, rarement en live et ont un assez bon ratio de gain.  


Un tel modèle permet de predire, suivant les actions effectuées par le client en 15 jours, potentiel $\text{lifetime value}$ d'un client.

Un tel model est sujet a etre amelioré mais cette piste peut servir de base solide pour la suite.  
En temps normal j'utilise la librairie "pickle" pour sauvegarder mes modeles mais pour ne pas generer de fichier suplementaire sur vos machines, j'ai retiré toute sauvegarde.  
Tous les modeles ont une seed fixe pour etre reproductible.
